In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.geometry import Point, Polygon

In [2]:
postmates_lowerManhattan = pd.read_csv('/Users/baileyrusso/PycharmProjects/GrubHubScraper/test7.csv',header=None)
postmates_lowerManhattan.columns = ['Name', 'Category', 'Favorites', 'Address', 'MenuItem', 'MenuItemDescription','MenuItemPrice']

postmates_midtown = pd.read_csv('/Users/baileyrusso/PycharmProjects/GrubHubScraper/test8.csv',header=None)
postmates_midtown.columns = ['Name', 'Category', 'Favorites', 'Address', 'MenuItem', 'MenuItemDescription','MenuItemPrice']

postmates_chelsea = pd.read_csv('/Users/baileyrusso/PycharmProjects/GrubHubScraper/test9.csv',header=None)
postmates_chelsea.columns = ['Name', 'Category', 'Favorites', 'Address', 'MenuItem', 'MenuItemDescription','MenuItemPrice']
for col in postmates_chelsea.columns:
    postmates_chelsea[col] = list(map(lambda row: str(row).replace('b\'','').replace('\'','').rstrip().replace('\'','') if row != "b''" else np.nan, postmates_chelsea[col]))




# Clean Address Column

In [4]:
def cleanAddressCol(df):
    addressList = []
    for i in range(len(df.Address)):
        address = df.Address.iloc[i]
        if pd.notnull(address):
            try:
                addressList.append(address.split('\n')[1].replace('(','').replace(')','').rstrip())
            except:
                print('Exception Occured: Make sure col lengths match')
                addressList.append(address.split('\\n')[1].replace('(','').replace(')','').rstrip())
        else:
            addressList.append(np.nan)

    df.Address = addressList
    print('Clean Address - Done')

# Clean Favorites Column

In [5]:
def cleanFavoritesCol(df):
    df['Favorites'] = list(map(lambda row: str(row).replace('added to favorites', '').rstrip(), df['Favorites']))
    df['Favorites'] = list(map(lambda row: str(row).replace('k', '00').replace('.','') if pd.notnull(row) else row, df['Favorites']))
    df['Favorites'] = df['Favorites'].astype(float)
    df['Favorites'] = df['Favorites'].fillna(0)
    print('Clean Favorites - Done')

# Clean Menu Item Price Column

In [6]:
def cleanMenuItemPriceCol(df):
    df['MenuItemPrice'] = [list(
        map(lambda row: float(str(row).replace('$', '').replace(',', '')) if pd.notnull(row) else row,
            df['MenuItemPrice']))][0]
    print('Clean Menu Item Price - Done')       

In [7]:
def getAddressCoords(df):
    geolocator = Nominatim(user_agent= 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36')
    address_list = list(df.groupby(['Name','Address']).mean().reset_index()['Address'])
    address_coords = []

    i = 0
    for address in address_list:
        try:
            print(i)
            i = i + 1
            print(geolocator.geocode(address + str(', NYC,NY'))[1])
            address_coords.append(tuple(geolocator.geocode(address + str(', NYC,NY'))[1]))
        except:
            address_coords.append(np.nan)
            continue

    d = {'Address': address_list, 'Address_Coordinates': address_coords}
    addressInfo = pd.DataFrame(d)
    
    return addressInfo
    '''
    df = pd.merge(df, addressInfo, on='Address')

    df['Address_Coordinates'] = list(map(lambda row: str(row).replace("(", "").replace(")", "").split(","), df['Address_Coordinates']))
    df['Address_Coordinates'] = list(map(lambda row: tuple([float(i) for i in row if pd.notnull(i)]), df['Address_Coordinates']))
    df[['Latitude', 'Longitude']] = pd.DataFrame(df['Address_Coordinates'].tolist(),index=df.index)
    print('Get Address Coords - Done')
    '''



In [8]:
cleanAddressCol(postmates_lowerManhattan)
cleanFavoritesCol(postmates_lowerManhattan)
cleanMenuItemPriceCol(postmates_lowerManhattan)
addressInfo = getAddressCoords(postmates_lowerManhattan)

postmates_lowerManhattan = pd.merge(postmates_lowerManhattan, addressInfo, on='Address')
postmates_lowerManhattan['Address_Coordinates'] = list(map(lambda row: str(row).replace("(", "").replace(")", "").split(","), postmates_lowerManhattan['Address_Coordinates']))
postmates_lowerManhattan['Address_Coordinates'] = list(map(lambda row: tuple([float(i) for i in row if pd.notnull(i)]), postmates_lowerManhattan['Address_Coordinates']))
postmates_lowerManhattan[['Latitude', 'Longitude']] = pd.DataFrame(postmates_lowerManhattan['Address_Coordinates'].tolist(),index=postmates_lowerManhattan.index)
print('Get Address Coords - Done')

cleanAddressCol(postmates_midtown)
cleanFavoritesCol(postmates_midtown)
cleanMenuItemPriceCol(postmates_midtown)
addressInfo = getAddressCoords(postmates_midtown)

postmates_midtown = pd.merge(postmates_midtown, addressInfo, on='Address')
postmates_midtown['Address_Coordinates'] = list(map(lambda row: str(row).replace("(", "").replace(")", "").split(","), postmates_midtown['Address_Coordinates']))
postmates_midtown['Address_Coordinates'] = list(map(lambda row: tuple([float(i) for i in row if pd.notnull(i)]), postmates_midtown['Address_Coordinates']))
postmates_midtown[['Latitude', 'Longitude']] = pd.DataFrame(postmates_midtown['Address_Coordinates'].tolist(),index=postmates_midtown.index)
print('Get Address Coords - Done')

cleanAddressCol(postmates_chelsea)
cleanFavoritesCol(postmates_chelsea)
cleanMenuItemPriceCol(postmates_chelsea)
addressInfo = getAddressCoords(postmates_chelsea)

postmates_chelsea = pd.merge(postmates_chelsea, addressInfo, on='Address')
postmates_chelsea['Address_Coordinates'] = list(map(lambda row: str(row).replace("(", "").replace(")", "").split(","), postmates_chelsea['Address_Coordinates']))
postmates_chelsea['Address_Coordinates'] = list(map(lambda row: tuple([float(i) for i in row if pd.notnull(i)]), postmates_chelsea['Address_Coordinates']))
postmates_chelsea[['Latitude', 'Longitude']] = pd.DataFrame(postmates_chelsea['Address_Coordinates'].tolist(),index=postmates_chelsea.index)
print('Get Address Coords - Done')



Clean Address - Done
Clean Favorites - Done
Clean Menu Item Price - Done
0
(40.7133082, -74.0080416)
1
(40.7047017, -74.0088567)
2
(40.7137186, -73.998778)
3
(40.715801, -73.9992469)
4
(40.7214812, -74.0013728)
5
(40.7163736, -73.9999049)
6
(40.7218498, -73.9971334)
7
(40.7097615, -74.0088919)
8
(40.7048612, -74.0095599)
9
(40.7097449, -74.0070195)
10
(40.71950495, -74.00409816786006)
11
(40.722171349999996, -73.99636760810697)
12
(40.7207158, -73.9947851)
13
(40.7300122, -74.00283299273195)
14
(40.72210135, -73.99766773251471)
15
(40.723624349999994, -73.97893769999999)
16
(40.70343895, -74.01108522118858)
17
(40.721206, -73.996278)
18
(40.7149438, -74.01086851127356)
19
(40.7048612, -74.0095599)
20
(40.7166208, -73.9997625)
21
(40.7186316, -73.9893489)
22
(40.716357, -73.9991738)
23
(40.713865, -73.9987984)
24
(40.709584199999995, -74.007034739265)
25
(40.707665750000004, -74.00595246337208)
26
(40.7215851, -73.9972327)
27
(40.7113535, -73.9998293)
28
(40.7048612, -74.0095599)
29
(40

75
(40.75945325, -73.96906294833789)
76
(40.7549721, -73.9655837)
77
(40.7501819, -73.9815291)
78
(40.7217736, -73.9854765)
79
(40.75945325, -73.96906294833789)
80
(40.75350042028495, -73.96630428877413)
81
(40.74407735, -73.97489924721782)
82
(40.755742850000004, -73.99654634999999)
83
(40.75334205, -73.97397388850807)
84
(40.7549721, -73.9655837)
85
(40.7441181, -73.9952986)
86
(40.7627719, -73.9966632)
87
(40.6994582, -73.9921983)
88
(40.7627719, -73.9966632)
89
(40.7549721, -73.9655837)
90
(40.7549721, -73.9655837)
91
(40.65874844444444, -74.00589496296297)
92
(40.7638948, -73.9921449)
93
(40.7640651, -73.9670374)
94
(40.7640822, -73.9887421)
95
(40.72589, -74.0092948)
96
(40.755742850000004, -73.99654634999999)
97
(40.7606764, -73.997433)
98
(40.7627719, -73.9966632)
99
(40.7549721, -73.9655837)
100
(40.7588045, -73.99648829750845)
101
(40.7583497, -73.9831961757719)
102
(40.65987526530612, -73.99836302040816)
103
(40.7640822, -73.9887421)
104
(40.75945325, -73.96906294833789)
105

Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occ

Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occ

Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occ

...

Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occ

Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occ

Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occ

Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occured: Make sure col lengths match
Exception Occ

(40.7356362, -73.9940878)
98
(40.7209234, -74.0043457)
99
(40.7292263, -73.9869976)
100
(40.743508899999995, -74.00320570000001)
101
(40.73855308163265, -73.99351377551021)
102
(40.7356362, -73.9940878)
103
(40.7242279, -73.98831849937335)
104
(40.7435985, -73.9999581)
105
(40.7332975, -74.0031947)
106
(40.7299487, -74.0006561)
107
(40.7435985, -73.9999581)
108
(40.7441181, -73.9952986)
109
(40.7441181, -73.9952986)
110
(40.666315000000004, -73.98931171428572)
111
(40.7435985, -73.9999581)
112
(40.666315000000004, -73.98931171428572)
113
(40.7441181, -73.9952986)
114
(40.7465957, -73.9972647)
115
(40.747502714285716, -73.996758)
116
(40.7460286, -74.0014102)
117
(40.743508899999995, -74.00320570000001)
118
(40.747502714285716, -73.996758)
119
(40.74103555, -73.99370340745092)
120
(40.666315000000004, -73.98931171428572)
121
(40.7465957, -73.9972647)
122
(40.7430204, -74.0036128)
123
(40.7439385, -73.9997311)
124
(40.82324905479452, -73.8086090890411)
125
(40.7441181, -73.9952986)
126
(

In [9]:
postmates = pd.concat([postmates_lowerManhattan, postmates_midtown, postmates_chelsea])
postmates.columns = ['Name', 'Category', 'Favorites', 'Address', 'MenuItem', 'MenuItemDescription', 'MenuItemPrice', 'Address_Coordinates','Latitude','Longitude']



In [10]:
postmates

,Name,Category,Favorites,Address,MenuItem,MenuItemDescription,MenuItemPrice,Address_Coordinates,Latitude,Longitude
0,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,BL Burrito (D),"Scrambled eggs, bacon, avocado, feta, crispy p...",15.4,"(40.7207158, -73.9947851)",40.720716,-73.994785
1,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,BL Burrito (D),"Scrambled eggs, bacon, avocado, feta, crispy p...",15.4,"(40.7207158, -73.9947851)",40.720716,-73.994785
2,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Avocado Smash (D),"Our classic avo smash with feta, heirloom toma...",14.3,"(40.7207158, -73.9947851)",40.720716,-73.994785
3,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Avocado Smash (D),"Our classic avo smash with feta, heirloom toma...",14.3,"(40.7207158, -73.9947851)",40.720716,-73.994785
4,Bluestone Lane Cafe,BREAKFAST,52.0,19 KENMARE ST,Iced Latte,Double shot of our signature maverick espresso...,NaN,"(40.7207158, -73.9947851)",40.720716,-73.994785
...,...,...,...,...,...,...,...,...,...,...
54464,The Triple Crown Ale House & Restaurant,PUB,0.0,330 7TH AVE,Corned Beef & Cabbage,"Housemade corned beef, boiled cabbage, steamed...",15.0,"(40.7476933, -73.9932597)",40.747693,-73.993260
54465,The Triple Crown Ale House & Restaurant,PUB,0.0,330 7TH AVE,Fish & Chips,"Guinness battered codfish, hand-cut fries, tar...",15.0,"(40.7476933, -73.9932597)",40.747693,-73.993260
54466,The Triple Crown Ale House & Restaurant,PUB,0.0,330 7TH AVE,Irish Beef Stew,"Tender beef, carrots, peas, garlic mashed pota...",15.0,"(40.7476933, -73.9932597)",40.747693,-73.993260
54467,The Triple Crown Ale House & Restaurant,PUB,0.0,330 7TH AVE,Roasted Atlantic Salmon,"Saut\xc3\xa9ed cabbage, roasted garlic, jasmin...",20.0,"(40.7476933, -73.9932597)",40.747693,-73.993260


In [11]:
postmates = postmates.drop_duplicates(subset=['Name','MenuItem','MenuItemDescription','Address_Coordinates'], keep='first')

In [12]:
postmates.to_csv('postmates_all2.csv')